# Aprendizado Federado para reconhecimento de contexto em dispositivos móveis
## Experimento 1 - Pré-treinamento usual com uma base de usuários e Aprendizado Federado embarcado partindo do modelo pretreinado 

In [1]:
%pip install scikit-multilearn

You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import es_utils as utils

2022-07-10 20:52:10.529068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: /usr/lib/x86_64-linux-gnu/libcuda.so.1: file too short; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-07-10 20:52:10.529089: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-10 20:52:10.529104: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-07-10 20:52:10.529251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


Inicialmente precisamos separar quais 'labels' usaremos. Aqui, optamos pelas 51 labels que segundo artigo do ExtraSensosy utiliza, de modo a ter bons parâmetros de comparação

In [4]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

"labels = ['label:LYING_DOWN',\n 'label:SITTING',\n 'label:FIX_walking',\n 'label:FIX_running',\n 'label:BICYCLING',\n 'label:SLEEPING',\n 'label:LAB_WORK',\n 'label:IN_CLASS',\n 'label:IN_A_MEETING',\n 'label:LOC_main_workplace',\n 'label:OR_indoors',\n 'label:OR_outside',\n 'label:IN_A_CAR',\n 'label:ON_A_BUS',\n 'label:DRIVE_-_I_M_THE_DRIVER',\n 'label:DRIVE_-_I_M_A_PASSENGER',\n 'label:LOC_home',\n 'label:FIX_restaurant',\n 'label:PHONE_IN_POCKET',\n 'label:OR_exercise',\n 'label:COOKING',\n 'label:SHOPPING',\n 'label:STROLLING',\n 'label:DRINKING__ALCOHOL_',\n 'label:BATHING_-_SHOWER',\n 'label:CLEANING',\n 'label:DOING_LAUNDRY',\n 'label:WASHING_DISHES',\n 'label:WATCHING_TV',\n 'label:SURFING_THE_INTERNET',\n 'label:AT_A_PARTY',\n 'label:AT_A_BAR',\n 'label:LOC_beach',\n 'label:SINGING',\n 'label:TALKING',\n 'label:COMPUTER_WORK',\n 'label:EATING',\n 'label:TOILET',\n 'label:GROOMING',\n 'label:DRESSING',\n 'label:AT_THE_GYM',\n 'label:STAIRS_-_GOING_UP',\n 'label:STAIRS_-_GOING

In [5]:
labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]

len(labels)

6

Separamos os 60 usuários em 5 pastas onde cada pasta considera 40 usuários selecionados aleatoriamente para treino e os 20 restantes para teste. Os dados dos 20 usuários são ainda separados em 4 arquivos *.csv* (x_train, x_test, y_train, y_test) e colocados em pastas nomeadas com o uuid do usuário.

In [9]:
#k_folds = utils.create_k_folds_n_users(5, 3, '/home/wander/OtherProjects/har_flower/sample_data')
k_folds = utils.create_k_folds_n_users(5, 40, '/home/nonroot/sample_data/')

/home/nonroot/es_utils.py:407: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:407: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:407: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:407: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:407: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

train_index {} [ 0  1  2  4  5  7  9 11 12 13 16 17 18 19 20 21 22 23 24 25 27 28 29 30
 31 32 34 35 36 37 38 39 40 41 42 43 46 47 48 49 51 52 53 54 55 57 58 59]
test_index {} [ 3  6  8 10 14 15 26 33 44 45 50 56]
split {} ['/home/nonroot/sample_data/61976C24-1C50-4355-9C49-AAE44A7D09F6.features_labels.csv']


/home/nonroot/es_utils.py:407: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:407: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:407: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:407: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fold_df_train = fold_df_train.append(pd.read_csv(csv))
/home/nonroot/es_utils.py:407: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

train_index {} [ 0  1  2  3  4  5  6  7  8 10 11 12 13 14 15 16 18 19 20 21 23 26 28 29
 30 31 32 33 34 35 36 37 43 44 45 46 47 48 49 50 51 52 54 55 56 57 58 59]
test_index {} [ 9 17 22 24 25 27 38 39 40 41 42 53]
split {} ['/home/nonroot/sample_data/61976C24-1C50-4355-9C49-AAE44A7D09F6.features_labels.csv']


Em seguida, para cada pasta, treinamos um modelo de MLP fazendo uma otimização de hiperparâmetros e convertemos em um modelo *tflite*. Os hiperparâmetros otimizados são:

- número de camadas escondidas (*hidden layers*): 1 ou 2
- número de neurônios em cada camada escondida: 4 a 64, com um passo de tamanho 4
- taxa de aprendizado (*learning rate*) do otimizador (*Adaptive Moment Estimation*, **Adam**): 1e-1, 1e-2, 1e-3 ou 1e-4

In [5]:
bas = []
for fold in k_folds:
    config = {
        'df_path': k_folds[fold]['40'],
        'labels': labels
    }
    
    har = utils.HAR(config)
    #har.run()
    model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    
    har.mlp.model.save(f'model/saved_model_{fold}')
    converter = tf.lite.TFLiteConverter.from_saved_model(f'model/saved_model_{fold}') # path to the SavedModel directory
    tflite_model = converter.convert()
    with open(f'model/model_base_{fold}.tflite', 'wb') as f:
      f.write(tflite_model)

print(bas)    

INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json
INFO:tensorflow:Reloading Tuner from hypertunning/experimento_1_RandSearchCV/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
4020/4020 [==============================] - 5s 967us/step - loss: 0.1990 - avg_multilabel_BA_2: 0.7295 - val_loss: 0.1695 - val_avg_multilabel_BA_2: 0.7836
Epoch 2/100
4020/4020 [==============================] - 4s 901us/step - loss: 0.1636 - avg_multilabel_BA_2: 0.7966 - val_loss: 0.1568 - val_avg_multilabel_BA_2: 0.8051
Epoch 3/100
4020/4020 [==============================] - 4s 881us/step - loss: 0.1559 - avg_multilabel_BA_2: 0.8104 - val_loss: 0.1537 - val_avg_multilabel_BA_2: 0.8144
Epoch 4/100
4020/4020 [==============================] - 4s 884us/step - loss: 0.1520 - avg_multilabel_BA_2: 0.8174 - val_loss: 0.1485 - val_avg_multilabel_BA_2: 0.8202
Epoch 5/100
4020/4020 [==============================] - 4s 881us/step - loss: 0.1492 - av

2022-04-11 16:23:54.850044: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: model/saved_model_fold_0/assets


2022-04-11 16:23:55.182829: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2022-04-11 16:23:55.182848: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-04-11 16:23:55.182852: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-04-11 16:23:55.183479: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model_fold_0
2022-04-11 16:23:55.184487: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-04-11 16:23:55.184499: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model_fold_0
2022-04-11 16:23:55.186853: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-11 16:23:55.217143: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: mod

INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json


...

INFO:tensorflow:Oracle triggered exit


Epoch 1/100
3847/3847 [==============================] - 4s 948us/step - loss: 0.1978 - avg_multilabel_BA_2: 0.8653 - val_loss: 0.1741 - val_avg_multilabel_BA_2: 0.8653
Epoch 2/100
3847/3847 [==============================] - 3s 870us/step - loss: 0.1635 - avg_multilabel_BA_2: 0.8652 - val_loss: 0.1596 - val_avg_multilabel_BA_2: 0.8652
Epoch 3/100
3847/3847 [==============================] - 3s 901us/step - loss: 0.1549 - avg_multilabel_BA_2: 0.8652 - val_loss: 0.1537 - val_avg_multilabel_BA_2: 0.8652
Epoch 4/100
3847/3847 [==============================] - 4s 921us/step - loss: 0.1501 - avg_multilabel_BA_2: 0.8652 - val_loss: 0.1488 - val_avg_multilabel_BA_2: 0.8651
Epoch 5/100
3847/3847 [==============================] - 3s 900us/step - loss: 0.1468 - avg_multilabel_BA_2: 0.8651 - val_loss: 0.1467 - val_avg_multilabel_BA_2: 0.8651
Epoch 6/100
3847/3847 [==============================] - 4s 917us/step - loss: 0.1441 - avg_multilabel_BA_2: 0.8651 - val_loss: 0.1440 - val_avg_multilabel

INFO:tensorflow:Assets written to: model/saved_model_fold_4/assets
2022-04-11 17:03:56.106726: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.


[0.8625562, 0.86519814, 0.86450756, 0.86537576, 0.8649584]


2022-04-11 17:03:56.106745: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2022-04-11 17:03:56.106750: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:372] Ignored change_concat_input_ranges.
2022-04-11 17:03:56.106925: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: model/saved_model_fold_4
2022-04-11 17:03:56.107694: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2022-04-11 17:03:56.107706: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: model/saved_model_fold_4
2022-04-11 17:03:56.109878: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-04-11 17:03:56.136549: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: model/saved_model_fold_4
2022-04-11 17:03:56.145208: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serv

In [6]:
har.data.x.head()

,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,...,lf_measurements:temperature_ambient,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label_source
0,1.057536,0.040597,-0.048977,0.124759,1.053158,1.057091,1.060935,0.344809,6.683838,5.043598,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2
1,1.057436,0.006165,-0.009415,0.018645,1.055086,1.057279,1.060143,1.014093,6.684595,5.042748,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2
2,1.056344,0.006302,-0.004635,0.013525,1.053282,1.056208,1.059165,1.429112,6.684594,5.043642,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2
3,1.056874,0.004767,-0.002796,0.007088,1.053958,1.057010,1.059996,2.190168,6.684602,5.043075,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2
4,1.057353,0.005415,0.006585,0.010781,1.054161,1.057100,1.059976,1.827865,6.684599,5.043392,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2


In [7]:
har.mlp.model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 44)                9988      
                                                                 
 dense_39 (Dense)            (None, 6)                 270       
                                                                 
Total params: 10,258
Trainable params: 10,258
Non-trainable params: 0
_________________________________________________________________


Com os modelos salvos, o restante do experimento segue em ambiente Android